# OPENTIS
## An Example with Alanine

Import Modules

In [1]:
import numpy as np
import mdtraj as md
import pandas as pd
import time
import sys, os
 
# in principle, all of these imports should be simplified once this is a package
from openpathsampling.orderparameter import OP_Function, OP_Volume, OP_MD_Function
from openpathsampling.snapshot import Snapshot
from openpathsampling.volume import LambdaVolumePeriodic, VolumeFactory as vf
from openpathsampling.pathmover import PathMoverFactory as mf
from openpathsampling.ensemble import EnsembleFactory as ef
from openpathsampling.ensemble import (LengthEnsemble, SequentialEnsemble, OutXEnsemble,
                      InXEnsemble, HitXEnsemble, LeaveXEnsemble)
from openpathsampling.storage import Storage
from openpathsampling.trajectory import Trajectory
from openpathsampling.calculation import Bootstrapping
from openpathsampling.shooting import UniformSelector
from openpathsampling.pathmover import PathMover
import simtk.unit as u

from openpathsampling.openmm_engine import OpenMMEngine

from openpathsampling.storage import Storage
from openpathsampling.visualize import PathTreeBuilder
from IPython.display import SVG

Radians to Degree conversion

In [2]:
degrees = 180/3.14159 # psi reports in radians; I think in degrees

Create an AlanineOpenMMSimulator for demonstration purposes

### Set simulation options and create a simulator object

In [3]:
options = {
            'temperature' : 300.0 * u.kelvin,
            'collision_rate' : 1.0 / u.picoseconds,
            'timestep' : 2.0 * u.femtoseconds,
            'nsteps_per_frame' : 10,
            'n_frames_max' : 5000,
            'start_time' : time.time(),
            'fn_initial_pdb' : "../data/Alanine_solvated.pdb",
            'platform' : 'fastest',
            'solute_indices' : range(22),
            'forcefield_solute' : 'amber96.xml',
            'forcefield_solvent' : 'tip3p.xml'
           }

In [4]:
engine = OpenMMEngine.auto(
                filename="trajectory.nc",
                template="../data/Alanine_solvated.pdb",
                options=options,
                mode='create'
                )

And save the storage in a separate variable for convenience

In [5]:
storage = engine.storage
PathMover.engine = engine

Equilibrate

In [6]:
engine.current_snapshot = engine.template
engine.equilibrate(5)
engine.initialized = True

Save equilibrated snapshot with index 0

In [7]:
storage.save(engine.current_snapshot);

### Order Parameters

this generates an order parameter (callable) object named psi (so if we call `psi(trajectory)` we get a list of the values of psi for each frame in the trajectory). This particular order parameter uses mdtraj's compute_dihedrals function, with the atoms in psi_atoms

In [8]:
psi_atoms = [6,8,14,16]
psi = OP_MD_Function("psi", md.compute_dihedrals,
                  indices=[psi_atoms])

phi_atoms = [4,6,8,14]
phi = OP_MD_Function("phi", md.compute_dihedrals,
                  indices=[phi_atoms])

storage.save([psi, phi]);

### Volumes

This creates two states using a one-dimensional order parameter (called _Lambda_ in TIS terminology). A snapshot is in the State as long as the order parameter is with specific bounds. 

In [9]:
stateA = LambdaVolumePeriodic(
    orderparameter=psi, 
    lambda_min=-120.0/degrees, 
    lambda_max=-30.0/degrees, 
    period_min=-180.0/degrees, 
    period_max=+180.0/degrees
)
stateB = LambdaVolumePeriodic(psi, 100/degrees, 180/degrees, -180.0/degrees, +180.0/degrees)

Now do the same for a set of lambda ranges to produce nested volumes.

In [10]:
minima = map((1.0 / degrees).__mul__,
             [-125, -135, -140, -142.5, -145.0, -147.0, -150.0])

maxima = map((1.0 / degrees).__mul__,
             [-25.0, -21.0, -18.5, -17.0, -15.0, -10.0, 0.0])

volume_set = vf.LambdaVolumePeriodicSet(psi, minima, maxima, -180.0/degrees, +180.0/degrees)

### Ensembles

Now do this automatically for all ensembles

In [11]:
interface0 = volume_set[0]
interface_set = ef.TISEnsembleSet(stateA, stateA | stateB, volume_set)
for no, interface in enumerate(interface_set):
    # Give each interface a name
    interface.name = 'Interface '+str(no)
    # And save all of these
    storage.ensemble.save(interface)

And create a special ensemble, that will create a first trajectory in the innermost TIS ensemble independent from where we start

The idea is to describe a trajectory type by a sequence of positions. First can be outside of stateA or not, then be inside stateA, etc...

In [12]:
    first_traj_ensemble = SequentialEnsemble([
        OutXEnsemble(stateA) | LengthEnsemble(0),
        InXEnsemble(stateA),
        (OutXEnsemble(stateA) & InXEnsemble(interface0)) | LengthEnsemble(0),
        InXEnsemble(interface0) | LengthEnsemble(0),
        OutXEnsemble(interface0),
        OutXEnsemble(stateA) | LengthEnsemble(0),
        InXEnsemble(stateA) & LengthEnsemble(1)
    ])

### start path generation

so lets try and see if we can generate a first path

load the initial snapshot (although we still have it) and generate using the Alanine simulator. The second option specifies a function : trajectory -> bool that keeps the simulation running as long as it is true. Our goal was to generate a path that belongs to a specific ensemble, so we use `forward` to determine if it makes sense to keep running or if the result cannot belong to the ensemble anymore.

In [13]:
snapshot = storage.load(Snapshot, 0)
total_path = engine.generate(
    snapshot= snapshot, 
    running = [first_traj_ensemble.can_append]
)

Show the length

In [14]:
print "Total trajectory length: ", len(total_path)

Total trajectory length:  30


And save the trajetory completely

In [15]:
storage.save(total_path);

Split the trajectory into parts that belong to the TIS ensemble (not the one we generated)

In [16]:
interface0_ensemble = interface_set[0]
segments = interface0_ensemble.split(total_path)

print "Traj in first_traj_ensemble? (should be)", 
print first_traj_ensemble(total_path)

print "Traj in TIS ensemble? (probably not)", 
print interface0_ensemble(total_path)


print "Number of segments in TIS ensemble: ", len(segments)
if len(segments):
    print "Length of each segment:"
    for i in range(len(segments)):
        print "  seg[{0}]: {1}".format(i, len(segments[i]))

Traj in first_traj_ensemble? (should be) True
Traj in TIS ensemble? (probably not) False
Number of segments in TIS ensemble:  1
Length of each segment:
  seg[0]: 9


Show some results and check if this worked

In [17]:
data = []
for frame in total_path:
    data.append((phi(frame)[0]*degrees, psi(frame)[0]*degrees, stateA(frame), interface0(frame), stateB(frame), first_traj_ensemble.can_append(total_path[slice(0,total_path.index(frame)+1)])))
    
dataframe = pd.DataFrame(data, columns=['phi', 'psi', 'stateA', 'interface0', 'stateB', 'appendable'])
dataframe[[0,1,2,3,4,5]].ix[[0,1,2,len(dataframe)-3,len(dataframe)-2,len(dataframe)-1]]

,phi,psi,stateA,interface0,stateB,appendable
0,-80.208523,-10.277041,[False],[False],[False],True
1,-83.716210,-10.580472,[False],[False],[False],True
2,-80.500104,-11.808651,[False],[False],[False],True
27,-84.539228,-24.238009,[False],[False],[False],True
28,-75.649586,-28.127798,[False],[True],[False],True
29,-74.051642,-33.646104,[True],[True],[False],False


In [18]:
print "Do our segments satisfy the ensemble?",
for seg in segments:
    print interface0_ensemble(seg),

Do our segments satisfy the ensemble? True


In [19]:
data = []
for frame in segments[0]:
    data.append((phi(frame)[0]*degrees, psi(frame)[0]*degrees, stateA(frame), interface0(frame), stateB(frame), first_traj_ensemble.can_append(total_path[slice(0,total_path.index(frame)+1)])))
    
dataframe = pd.DataFrame(data, columns=['phi', 'psi', 'stateA', 'interface0', 'stateB', 'appendable'])
dataframe[[0,1,2,3,4,5]]

,phi,psi,stateA,interface0,stateB,appendable
0,-66.731203,-31.589655,[True],[True],[False],True
1,-71.096592,-28.818430,[False],[True],[False],True
2,-68.044198,-29.870761,[False],[True],[False],True
3,-70.683864,-29.604845,[False],[True],[False],True
4,-79.314498,-22.405249,[False],[False],[False],True
5,-75.532025,-26.627549,[False],[True],[False],True
6,-84.539228,-24.238009,[False],[False],[False],True
7,-75.649586,-28.127798,[False],[True],[False],True
8,-74.051642,-33.646104,[True],[True],[False],False


### Bootstrapping

Run a bootstrapping (not TIS) simulation that shoots from an ensemble until the next interface is reached then switch to the next ensemble to drive the system out of stateA

In [20]:
mover_set = mf.OneWayShootingSet(UniformSelector(), interface_set)

In [21]:
bootstrap = Bootstrapping(storage=storage,
                          engine=engine,
                          ensembles=interface_set,
                          movers=mover_set,
                          trajectory=segments[0])

Run Bootstrapping with at most 50 attempts per ensemble

In [22]:
bootstrap.run(300)

Save all computed phi/psi values which depends on whether they have been needed before

In [27]:
psi(storage.snapshot)

837 837 837
[672, 387, 544, 619, 695, 415, 423, 412, 494, 788, 551, 132, 517, 550, 799, 516, 536, 787, 416, 585, 506, 457, 206, 757, 499, 205, 484, 133, 760, 367, 728, 368, 795, 417, 351, 556, 557, 662, 558, 160, 559, 830, 134, 458, 424, 575, 789, 369, 673, 737, 434, 407, 796, 460, 689, 135, 562, 617, 370, 693, 345, 419, 698, 121, 410, 120, 492, 208, 123, 469, 207, 199, 563, 371, 785, 394, 420, 821, 652, 729, 448, 427, 124, 421, 812, 663, 137, 380, 588, 690, 647, 831, 136, 744, 230, 590, 36, 574, 518, 125, 378, 674, 38, 403, 39, 40, 644, 269, 270, 565, 271, 545, 142, 814, 245, 237, 272, 537, 769, 561, 333, 514, 783, 815, 6, 765, 822, 273, 435, 656, 236, 334, 363, 664, 791, 816, 346, 810, 631, 832, 274, 335, 144, 576, 5, 713, 675, 238, 626, 817, 665, 615, 483, 145, 818, 753, 239, 524, 807, 306, 288, 362, 364, 24, 572, 146, 571, 474, 344, 235, 7, 786, 502, 512, 823, 819, 538, 377, 520, 655, 526, 528, 711, 8, 504, 465, 530, 366, 531, 833, 240, 761, 352, 477, 533, 643, 275, 319, 676, 552, 

[-0.1793680489063263,
 array([-0.17894943], dtype=float32),
 -0.18466392159461975,
 -0.20609967410564423,
 -0.21282809972763062,
 -0.20899459719657898,
 -0.2316381335258484,
 -0.278094619512558,
 -0.3789004981517792,
 -0.37670737504959106,
 -0.47317102551460266,
 -0.4910108745098114,
 -0.5396372675895691,
 -0.593560516834259,
 -0.5942068696022034,
 -0.49879318475723267,
 -0.57906574010849,
 -0.5206391215324402,
 -0.5436614751815796,
 -0.5222721695899963,
 -0.5240384340286255,
 -0.5301477313041687,
 -0.5513430237770081,
 -0.5029760599136353,
 -0.5213426947593689,
 -0.5167015790939331,
 -0.3910450339317322,
 -0.46473801136016846,
 -0.4230327010154724,
 -0.4909222722053528,
 -0.5872347950935364,
 -0.5688533186912537,
 -0.36332225799560547,
 -0.4819638133049011,
 -0.4230327010154724,
 -0.5876132249832153,
 -0.5103480815887451,
 -0.4803977906703949,
 -0.5120212435722351,
 -0.49881231784820557,
 -0.42993661761283875,
 -0.3720826208591461,
 -0.44116947054862976,
 -0.38473010063171387,
 -0.459

In [28]:
len([p for p in list(set(storage.snapshot))])

838

In [29]:
psi.cache_dict.clear()
psi.post.post.post.post.nod_stores
storage.cv.sync()
#print psi.post.post.post.post.nod_stores.itervalues().next()
#print storage.variables['orderparameter_0_psi_value'][:]
storage.sync()
print len(storage.snapshot)
print [snap.idx[storage] for snap in storage.snapshot]
print psi(storage.snapshot[143])

838
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 

In [32]:
storage.collectivevariable.sync(psi);
storage.collectivevariable.sync(phi);

Create an orderparameter from a volume which is just 1 or 0 and can thus be stored for later analysis

In [35]:
op_inA = OP_Volume('StateA', stateA)
op_inB = OP_Volume('StateB', stateB)
op_notinAorB = OP_Volume('StateX', ~ (stateA | stateB))

Compute the orderparameter for all snapshots

In [36]:
op_inA(storage.snapshot.all())
op_inB(storage.snapshot.all())
op_notinAorB(storage.snapshot.all());

Save the new orderparameters

In [37]:
storage.save([op_inA, op_inB, op_notinAorB]);

## Visualization

Create a PathTree generator

In [ ]:
tree = PathTreeBuilder(storage)
op_inA.name
storage.save(op_inA)
storage.cv.load(0)
op_inA.idx

Change the settings to show rejected pathways, mark OrderParaemters `stateA` and `stateX`, and show the 'psi' value as text inside of the boxes

In [ ]:
%prun [ psi(storage.snapshot[nn]) for nn in range(len(storage.snapshot))]

In [ ]:
%prun psi([ storage.snapshot[nn] for nn in range(len(storage.snapshot))])

In [ ]:
%prun psi(storage.snapshot)

In [ ]:
%prun list(storage.snapshot)

In [ ]:
tree.rejected = True
tree.states = []
# Some ideas for orderparameters to visualize
tree.op = lambda snap : 'B' if snap.reversed else 'F'
tree.op = storage.configuration.op_idx
tree.op = lambda snap : int(psi(snap)[0]/3.1415926 * 180)

tree.from_samples(tree.construct_heritage(storage, storage.sample.last()))

In [ ]:
op = storage.collectivevariable.load('StateA')
storage.cv.sync()
psi(storage.snapshot)
storage.sync()
print psi(storage.snapshot[2])
psi.cache_dict.clear()

In [ ]:
mat = tree._to_matrix()
for row in mat:
    for val in row:
        if val is not None:
            print val, bool(op(val))

Render the tree

In [ ]:
view = tree.renderer
view.zoom = 1.1
view.scale_y = 24
view.scale_x = 24
view.font_size = 0.35
SVG(view.to_svg())

An alternate view which is similar to the standard way of plotting

In [ ]:
tree.rejected = False
tree.states = []
tree.op = None
samples = storage.sample.by_ensemble(storage.ensemble.load(4))
tree.from_samples(samples)
view = tree.renderer
view.zoom = 1.1
view.horizontal_gap = -0.01
view.scale_y = 15
view.scale_x = 24
view.font_size = 0.8
view.font_family = 'Times'
SVG(view.to_svg())

### Phi/Psi Plots

In [ ]:
# Imports for plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.legend_handler import HandlerLine2D

Make sure that all phi/psi values have been computed!

In [ ]:
phi(storage.snapshot.all());
psi(storage.snapshot.all());

In [ ]:
plt.figure(figsize=(5, 5))

for traj in storage.trajectory.iterator():
    phi_angles = degrees * np.array(phi(traj)).flatten()
    psi_angles = degrees * np.array(psi(traj)).flatten()
    plt.plot(phi_angles, psi_angles, 'ro', linewidth=1);
    
plt.xlim(-180, 180);
plt.ylim(-180, 180);

In [ ]:
plt.figure(figsize=(8, 8))

traj = storage.trajectory.last()
phi_angles = degrees * np.array(phi(traj)).flatten()
psi_angles = degrees * np.array(psi(traj)).flatten()
plt.plot(phi_angles, psi_angles, 'k-', linewidth=1);

for snapshot in traj:
    phi_angles = degrees * np.array(phi(snapshot)).flatten()
    psi_angles = degrees * np.array(psi(snapshot)).flatten()
    if op_notinAorB(snapshot):
        plt.plot(phi_angles, psi_angles, 'ro', linewidth=1);
    else:
        plt.plot(phi_angles, psi_angles, 'bo', linewidth=1);
        
plt.xlim(-180, 180);
plt.ylim(-180, 180);

In [ ]:
plt.figure(figsize=(8, 8))

traj = storage.sample.last().trajectory
phi_angles = degrees * np.array(phi(traj)).flatten()
psi_angles = degrees * np.array(psi(traj)).flatten()
plt.plot(phi_angles, psi_angles, 'k-', linewidth=1);


#for snapshot in simulator.storage.snapshot.iterator():
for snapshot in traj:
    phi_angles = degrees * np.array(phi(snapshot)).flatten()
    psi_angles = degrees * np.array(psi(snapshot)).flatten()
    if op_notinAorB(snapshot):
        plt.plot(phi_angles, psi_angles, 'ro', linewidth=1);
    else:
        plt.plot(phi_angles, psi_angles, 'bo', linewidth=1);
        
plt.xlim(-180, 180);
plt.ylim(-180, 180);

In [ ]:
storage.sync()